# OBJETIVO

Análise de % de Admissões com CID preenchido -> Excluir SEM ALTA e afins -> (tanto junto, quanto separado, ter visão geral do hosp e por nome do médico)

Ter noções evolutivas e identificar os médicos que não estão preenchendo os CIDs

Criar um PPT com os resultados / análises dos dados extraídos

# PASSO A PASSO

Ler Bases

Calcular o % agrupando por ano, mês, hospital e médico 

# LER BASES

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from dateutil.relativedelta import relativedelta

# HOSPITAL 1

In [26]:
# PRODUÇÃO GERAL

# Especificar o caminho da pasta
folder_path = r" caminho da pasta para leitura de bases / arquivos excel e csv "

# Inicializar um DataFrame vazio
df_producaoCBV = pd.DataFrame()

# Loop através de todos os arquivos na pasta
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    try:
        if file_name.endswith(('.xlsx', '.xls')):
            # Ler arquivo Excel
            df_temp = pd.read_excel(file_path)
        elif file_name.endswith('.csv'):
            # Tentar detectar automaticamente o delimitador
            df_temp = pd.read_csv(file_path, sep=None, engine='python')
        else:
            # Pular arquivos que não sejam Excel ou CSV
            continue
        
        # Concatenar os dados lidos com o DataFrame principal
        df_producaoCBV = pd.concat([df_producaoCBV, df_temp], ignore_index=True)
    except Exception as e:
        print(f"Erro ao processar o arquivo {file_name}: {e}")

df_producaoCBV = df_producaoCBV.drop_duplicates()
df_producaoCBV['Cód. Paciente'] = df_producaoCBV['Cód. Paciente'].astype(str)
df_producaoCBV['Cód. Paciente'] = df_producaoCBV['Cód. Paciente'].str.rstrip('.0')

df_producaoCBV['Cód. Admissão'] = df_producaoCBV['Cód. Admissão'].astype(str)
df_producaoCBV['Cód. Admissão'] = df_producaoCBV['Cód. Admissão'].str.rstrip('.0')

C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
df_producaoCBV.head(100)

In [28]:
df_producaoCBV['Cód. Admissão'].nunique()

92530

# TRATAR / FILTRAR DADOS -> HOSPITAL 1

In [29]:
df_producaoCBV.columns

Index(['Ano', 'Mês', 'Cód. Admissão', 'Data Admissão', 'Hora Admissão',
       'Categoria', 'Subcategoria', 'Profissional Solicitante',
       'Profissional Admissão', 'Médico Externo', 'Classificação Produto',
       'CID da Alta', 'Tipo Recebimento', 'Convênio', 'Plano',
       'Perfil Particular', 'Procedimento Principal', 'Produto Lançamento',
       'Cód. Apresentação', 'Unidade', 'Especialidade', 'Caráter Admissão',
       'Tipo Paciente', 'Observação Admissão', 'Tipo Produto', 'Pacote',
       'Cód. Paciente', 'Paciente', 'Data Nascimento', 'Idade', 'Faixa Etária',
       'Destino', 'Qtd. Produzida', 'Valor Produzido', 'Solicitante', 'Médico',
       'Cirurgião'],
      dtype='object')

In [30]:
df_producaoCBV2 = df_producaoCBV.copy()

In [31]:
# TIRAR COLUNAS QUE NÃO VÃO SER UTILIZADAS
df_producaoCBV2 = df_producaoCBV2[['Cód. Paciente', 'Cód. Admissão', 'Ano', 'Mês', 'Médico', 'Unidade', 'CID da Alta']]

# RENOMAR COLUNA DE UNIDADE PARA NOME ÚNICO DO HOSPITAL
df_producaoCBV2 = df_producaoCBV2.rename(columns={'Unidade': 'Hospital'})
df_producaoCBV2['Hospital'] = 'HOSPITAL 1'

In [ ]:
df_producaoCBV2.head(100)

# VISÃO -> ANO, MÊS, HOSP -> HOSPITAL 1

# % TOTAL DO HOSP NO TEMPO -> HOSPITAL 1

In [33]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_cbv_AMH = df_producaoCBV2[
    (df_producaoCBV2['CID da Alta'].notna()) & 
    (df_producaoCBV2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_cbv_AMH = df_producaoCBV2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_cbv_AMH = df_cid_preenchido_cbv_AMH.groupby(
    ['Ano', 'Mês', 'Hospital']
).agg(
    adm_com_cid_cbv_AMH=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_cbv_AMH = df_cid_total_cbv_AMH.groupby(
    ['Ano', 'Mês', 'Hospital']
).agg(
    adm_total_cbv_AMH=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoCBV_AMH = pd.merge(
    agrupado_total_cbv_AMH, 
    agrupado_cid_cbv_AMH, 
    on=['Ano', 'Mês', 'Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoCBV_AMH['% Admissões Com CID Preenchido'] = (
    (df_resultadoCBV_AMH['adm_com_cid_cbv_AMH'] / df_resultadoCBV_AMH['adm_total_cbv_AMH']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoCBV_AMH = df_resultadoCBV_AMH.rename(columns={'adm_total_cbv_AMH': 'Total Admissões'})
df_resultadoCBV_AMH = df_resultadoCBV_AMH.rename(columns={'adm_com_cid_cbv_AMH': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoCBV_AMH['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoCBV_AMH['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoCBV_AMH.head(100)

# BAIXAR BASE / DF -> VISÃO ANO, MÊS, HOSP -> HOSPITAL 1

In [36]:
df_resultadoCBV_AMH.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> HOSP -> HOSPITAL 1

# % TOTAL DO HOSP -> HOSPITAL 1

In [37]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_cbv_H = df_producaoCBV2[
    (df_producaoCBV2['CID da Alta'].notna()) & 
    (df_producaoCBV2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_cbv_H = df_producaoCBV2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_cbv_H = df_cid_preenchido_cbv_H.groupby(
    ['Hospital']
).agg(
    adm_com_cid_cbv_H=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_cbv_H = df_cid_total_cbv_H.groupby(
    ['Hospital']
).agg(
    adm_total_cbv_H=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoCBV_H = pd.merge(
    agrupado_total_cbv_H, 
    agrupado_cid_cbv_H, 
    on=['Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoCBV_H['% Admissões Com CID Preenchido'] = (
    (df_resultadoCBV_H['adm_com_cid_cbv_H'] / df_resultadoCBV_H['adm_total_cbv_H']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoCBV_H = df_resultadoCBV_H.rename(columns={'adm_total_cbv_H': 'Total Admissões'})
df_resultadoCBV_H = df_resultadoCBV_H.rename(columns={'adm_total_cbv_H': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoCBV_H['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoCBV_H['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoCBV_H.head(100)

# BAIXAR BASE / DF -> VISÃO HOSP -> HOSPITAL 1

In [39]:
df_resultadoCBV_H.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> ANO, MÊS, HOSP E MÉDICO -> HOSPITAL 1

# % TOTAL DO HOSP NO TEMPO POR MÉDICO -> HOSPITAL 1

In [41]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_cbv_AMHM = df_producaoCBV2[
    (df_producaoCBV2['CID da Alta'].notna()) & 
    (df_producaoCBV2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_cbv_AMHM = df_producaoCBV2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_cbv_AMHM = df_cid_preenchido_cbv_AMHM.groupby(
    ['Ano', 'Mês', 'Médico','Hospital']
).agg(
    adm_com_cid_cbv_AMHM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_cbv_AMHM = df_cid_total_cbv_AMHM.groupby(
    ['Ano', 'Mês', 'Médico','Hospital']
).agg(
    adm_total_cbv_AMHM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoCBV_AMHM = pd.merge(
    agrupado_total_cbv_AMHM, 
    agrupado_cid_cbv_AMHM, 
    on=['Ano', 'Mês', 'Médico','Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoCBV_AMHM['% Admissões Com CID Preenchido'] = (
    (df_resultadoCBV_AMHM['adm_com_cid_cbv_AMHM'] / df_resultadoCBV_AMHM['adm_total_cbv_AMHM']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoCBV_AMHM = df_resultadoCBV_AMHM.rename(columns={'adm_total_cbv_AMHM': 'Total Admissões'})
df_resultadoCBV_AMHM = df_resultadoCBV_AMHM.rename(columns={'adm_com_cid_cbv_AMHM': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoCBV_AMHM['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoCBV_AMHM['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoCBV_AMHM.head(100)

# BAIXAR BASE / DF -> VISÃO ANO, MÊS, HOSP E MÉDICO -> HOSPITAL 1

In [45]:
df_resultadoCBV_AMHM.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> HOSP E MÉDICO -> HOSPITAL 1

# % TOTAL DO HOSP POR MÉDICO -> HOSPITAL 1

In [46]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_cbv_HM = df_producaoCBV2[
    (df_producaoCBV2['CID da Alta'].notna()) & 
    (df_producaoCBV2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_cbv_HM = df_producaoCBV2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_cbv_HM = df_cid_preenchido_cbv_HM.groupby(
    ['Médico','Hospital']
).agg(
    adm_com_cid_cbv_HM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_cbv_HM = df_cid_total_cbv_HM.groupby(
    ['Médico','Hospital']
).agg(
    adm_total_cbv_HM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoCBV_HM = pd.merge(
    agrupado_total_cbv_HM, 
    agrupado_cid_cbv_HM, 
    on=['Médico','Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoCBV_HM['% Admissões Com CID Preenchido'] = (
    (df_resultadoCBV_HM['adm_com_cid_cbv_HM'] / df_resultadoCBV_HM['adm_total_cbv_HM']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoCBV_HM = df_resultadoCBV_HM.rename(columns={'adm_total_cbv_HM': 'Total Admissões'})
df_resultadoCBV_HM = df_resultadoCBV_HM.rename(columns={'adm_com_cid_cbv_HM': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoCBV_HM['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoCBV_HM['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoCBV_HM.head(100)

# BAIXAR BASE / DF -> VISÃO HOSP E MÉDICO -> HOSPITAL 1

In [48]:
df_resultadoCBV_HM.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# ///////////////////////////////////////////////////////////////////////////////////////////

# ///////////////////////////////////////////////////////////////////////////////////////////

# ///////////////////////////////////////////////////////////////////////////////////////////

# HOSPITAL 2

In [49]:
# PRODUÇÃO GERAL

# Especificar o caminho da pasta
folder_path = r" caminho da pasta para leitura de bases / arquivos excel e csv "

# Inicializar um DataFrame vazio
df_producaoHOC = pd.DataFrame()

# Loop através de todos os arquivos na pasta
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    try:
        if file_name.endswith(('.xlsx', '.xls')):
            # Ler arquivo Excel
            df_temp = pd.read_excel(file_path)
        elif file_name.endswith('.csv'):
            # Tentar detectar automaticamente o delimitador
            df_temp = pd.read_csv(file_path, sep=None, engine='python')
        else:
            # Pular arquivos que não sejam Excel ou CSV
            continue
        
        # Concatenar os dados lidos com o DataFrame principal
        df_producaoHOC = pd.concat([df_producaoHOC, df_temp], ignore_index=True)
    except Exception as e:
        print(f"Erro ao processar o arquivo {file_name}: {e}")

df_producaoHOC = df_producaoHOC.drop_duplicates()
df_producaoHOC['Cód. Paciente'] = df_producaoHOC['Cód. Paciente'].astype(str)
df_producaoHOC['Cód. Paciente'] = df_producaoHOC['Cód. Paciente'].str.rstrip('.0')

df_producaoHOC['Cód. Admissão'] = df_producaoHOC['Cód. Admissão'].astype(str)
df_producaoHOC['Cód. Admissão'] = df_producaoHOC['Cód. Admissão'].str.rstrip('.0')

C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
df_producaoHOC.head(100)

In [50]:
df_producaoHOC['Cód. Admissão'].nunique()

117274

# TRATAR / FILTRAR DADOS -> HOSPITAL 2

In [54]:
df_producaoHOC.columns

Index(['Ano', 'Mês', 'Cód. Admissão', 'Data Admissão', 'Hora Admissão',
       'Categoria', 'Subcategoria', 'Profissional Solicitante',
       'Profissional Admissão', 'Médico Externo', 'Classificação Produto',
       'CID da Alta', 'Tipo Recebimento', 'Convênio', 'Plano',
       'Perfil Particular', 'Procedimento Principal', 'Produto Lançamento',
       'Cód. Apresentação', 'Unidade', 'Especialidade', 'Caráter Admissão',
       'Tipo Paciente', 'Observação Admissão', 'Tipo Produto', 'Pacote',
       'Cód. Paciente', 'Paciente', 'Data Nascimento', 'Idade', 'Faixa Etária',
       'Destino', 'Qtd. Produzida', 'Valor Produzido'],
      dtype='object')

In [59]:
df_producaoHOC2 = df_producaoHOC.copy()

In [60]:
# TIRAR COLUNAS QUE NÃO VÃO SER UTILIZADAS
df_producaoHOC2 = df_producaoHOC2[['Cód. Paciente', 'Cód. Admissão', 'Ano', 'Mês', 'Profissional Admissão', 'Unidade', 'CID da Alta']]

# RENOMAR COLUNA DE UNIDADE PARA NOME ÚNICO DO HOSPITAL
df_producaoHOC2 = df_producaoHOC2.rename(columns={'Unidade': 'Hospital'})
df_producaoHOC2['Hospital'] = 'HOSPITAL 2'

In [ ]:
df_producaoHOC2.head(100)

# VISÃO -> ANO, MÊS, HOSP -> HOSPITAL 2

# % TOTAL DO HOSP NO TEMPO -> HOSPITAL 2

In [66]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_hoc_AMH = df_producaoHOC2[
    (df_producaoHOC2['CID da Alta'].notna()) & 
    (df_producaoHOC2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_hoc_AMH = df_producaoHOC2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_hoc_AMH = df_cid_preenchido_hoc_AMH.groupby(
    ['Ano', 'Mês', 'Hospital']
).agg(
    adm_com_cid_hoc_AMH=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_hoc_AMH = df_cid_total_hoc_AMH.groupby(
    ['Ano', 'Mês', 'Hospital']
).agg(
    adm_total_hoc_AMH=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHOC_AMH = pd.merge(
    agrupado_total_hoc_AMH, 
    agrupado_cid_hoc_AMH, 
    on=['Ano', 'Mês', 'Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHOC_AMH['% Admissões Com CID Preenchido'] = (
    (df_resultadoHOC_AMH['adm_com_cid_hoc_AMH'] / df_resultadoHOC_AMH['adm_total_hoc_AMH']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoHOC_AMH = df_resultadoHOC_AMH.rename(columns={'adm_total_hoc_AMH': 'Total Admissões'})
df_resultadoHOC_AMH = df_resultadoHOC_AMH.rename(columns={'adm_com_cid_hoc_AMH': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHOC_AMH['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHOC_AMH['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHOC_AMH.head(100)

# BAIXAR BASE / DF -> VISÃO ANO, MÊS, HOSP -> HOSPITAL 2

In [68]:
df_resultadoHOC_AMH.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> HOSP -> HOSPITAL 2

# % TOTAL DO HOSP -> HOSPITAL 2

In [69]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_hoc_H = df_producaoHOC2[
    (df_producaoHOC2['CID da Alta'].notna()) & 
    (df_producaoHOC2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_hoc_H = df_producaoHOC2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_hoc_H = df_cid_preenchido_hoc_H.groupby(
    ['Hospital']
).agg(
    adm_com_cid_hoc_H=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_hoc_H = df_cid_total_hoc_H.groupby(
    ['Hospital']
).agg(
    adm_total_hoc_H=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHOC_H = pd.merge(
    agrupado_total_hoc_H, 
    agrupado_cid_hoc_H, 
    on=['Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHOC_H['% Admissões Com CID Preenchido'] = (
    (df_resultadoHOC_H['adm_com_cid_hoc_H'] / df_resultadoHOC_H['adm_total_hoc_H']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoHOC_H = df_resultadoHOC_H.rename(columns={'adm_total_hoc_H': 'Total Admissões'})
df_resultadoHOC_H = df_resultadoHOC_H.rename(columns={'adm_total_hoc_H': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHOC_H['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHOC_H['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHOC_H.head(100)

# BAIXAR BASE / DF -> VISÃO HOSP -> HOSPITAL 2

In [71]:
df_resultadoHOC_H.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> ANO, MÊS, HOSP E MÉDICO -> HOSPITAL 2

# % TOTAL DO HOSP NO TEMPO POR MÉDICO -> HOSPITAL 2

In [73]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_hoc_AMHM = df_producaoHOC2[
    (df_producaoHOC2['CID da Alta'].notna()) & 
    (df_producaoHOC2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_hoc_AMHM = df_producaoHOC2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_hoc_AMHM = df_cid_preenchido_hoc_AMHM.groupby(
    ['Ano', 'Mês', 'Profissional Admissão','Hospital']
).agg(
    adm_com_cid_hoc_AMHM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_hoc_AMHM = df_cid_total_hoc_AMHM.groupby(
    ['Ano', 'Mês', 'Profissional Admissão','Hospital']
).agg(
    adm_total_hoc_AMHM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHOC_AMHM = pd.merge(
    agrupado_total_hoc_AMHM, 
    agrupado_cid_hoc_AMHM, 
    on=['Ano', 'Mês', 'Profissional Admissão','Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHOC_AMHM['% Admissões Com CID Preenchido'] = (
    (df_resultadoHOC_AMHM['adm_com_cid_hoc_AMHM'] / df_resultadoHOC_AMHM['adm_total_hoc_AMHM']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoHOC_AMHM = df_resultadoHOC_AMHM.rename(columns={'adm_total_hoc_AMHM': 'Total Admissões'})
df_resultadoHOC_AMHM = df_resultadoHOC_AMHM.rename(columns={'adm_com_cid_hoc_AMHM': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHOC_AMHM['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHOC_AMHM['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHOC_AMHM.head(100)

# BAIXAR BASE / DF -> VISÃO ANO, MÊS, HOSP E MÉDICO -> HOSPITAL 2

In [75]:
df_resultadoHOC_AMHM.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> HOSP E MÉDICO -> HOSPITAL 2

# % TOTAL DO HOSP POR MÉDICO -> HOSPITAL 2

In [76]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_hoc_HM = df_producaoHOC2[
    (df_producaoHOC2['CID da Alta'].notna()) & 
    (df_producaoHOC2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_hoc_HM = df_producaoHOC2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_hoc_HM = df_cid_preenchido_hoc_HM.groupby(
    ['Profissional Admissão','Hospital']
).agg(
    adm_com_cid_hoc_HM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_hoc_HM = df_cid_total_hoc_HM.groupby(
    ['Profissional Admissão','Hospital']
).agg(
    adm_total_hoc_HM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHOC_HM = pd.merge(
    agrupado_total_hoc_HM, 
    agrupado_cid_hoc_HM, 
    on=['Profissional Admissão','Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHOC_HM['% Admissões Com CID Preenchido'] = (
    (df_resultadoHOC_HM['adm_com_cid_hoc_HM'] / df_resultadoHOC_HM['adm_total_hoc_HM']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoHOC_HM = df_resultadoHOC_HM.rename(columns={'adm_total_hoc_HM': 'Total Admissões'})
df_resultadoHOC_HM = df_resultadoHOC_HM.rename(columns={'adm_com_cid_hoc_HM': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHOC_HM['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHOC_HM['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHOC_HM.head(100)

# BAIXAR BASE / DF -> VISÃO HOSP E MÉDICO -> HOSPITAL 2

In [78]:
df_resultadoHOC_HM.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# ///////////////////////////////////////////////////////////////////////////////////////////

# ///////////////////////////////////////////////////////////////////////////////////////////

# ///////////////////////////////////////////////////////////////////////////////////////////

# HOSPITAL 3

In [79]:
# PRODUÇÃO GERAL

# Especificar o caminho da pasta
folder_path = r" caminho da pasta para leitura de bases / arquivos excel e csv "

# Inicializar um DataFrame vazio
df_producaoHRO = pd.DataFrame()

# Loop através de todos os arquivos na pasta
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    try:
        if file_name.endswith(('.xlsx', '.xls')):
            # Ler arquivo Excel
            df_temp = pd.read_excel(file_path)
        elif file_name.endswith('.csv'):
            # Tentar detectar automaticamente o delimitador
            df_temp = pd.read_csv(file_path, sep=None, engine='python')
        else:
            # Pular arquivos que não sejam Excel ou CSV
            continue
        
        # Concatenar os dados lidos com o DataFrame principal
        df_producaoHRO = pd.concat([df_producaoHRO, df_temp], ignore_index=True)
    except Exception as e:
        print(f"Erro ao processar o arquivo {file_name}: {e}")

df_producaoHRO = df_producaoHRO.drop_duplicates()
df_producaoHRO['Cód. Paciente'] = df_producaoHRO['Cód. Paciente'].astype(str)
df_producaoHRO['Cód. Paciente'] = df_producaoHRO['Cód. Paciente'].str.rstrip('.0')

df_producaoHRO['Cód. Admissão'] = df_producaoHRO['Cód. Admissão'].astype(str)
df_producaoHRO['Cód. Admissão'] = df_producaoHRO['Cód. Admissão'].str.rstrip('.0')

C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
df_producaoHRO.head(100)

In [81]:
df_producaoHRO['Cód. Admissão'].nunique()

142701

# TRATAR / FILTRAR DADOS -> HOSPITAL 3

In [82]:
df_producaoHRO.columns

Index(['Ano', 'Mês', 'Cód. Admissão', 'Data Admissão', 'Hora Admissão',
       'Categoria', 'Subcategoria', 'Profissional Solicitante',
       'Profissional Admissão', 'Médico Externo', 'Classificação Produto',
       'CID da Alta', 'Tipo Recebimento', 'Convênio', 'Plano',
       'Perfil Particular', 'Procedimento Principal', 'Produto Lançamento',
       'Cód. Apresentação', 'Unidade', 'Especialidade', 'Caráter Admissão',
       'Tipo Paciente', 'Observação Admissão', 'Tipo Produto', 'Pacote',
       'Cód. Paciente', 'Paciente', 'Data Nascimento', 'Idade', 'Faixa Etária',
       'Destino', 'Qtd. Produzida', 'Valor Produzido', 'Solicitante', 'Médico',
       'Cirurgião'],
      dtype='object')

In [83]:
df_producaoHRO2 = df_producaoHRO.copy()

In [84]:
# TIRAR COLUNAS QUE NÃO VÃO SER UTILIZADAS
df_producaoHRO2 = df_producaoHRO2[['Cód. Paciente', 'Cód. Admissão', 'Ano', 'Mês', 'Médico', 'Unidade', 'CID da Alta']]

# RENOMAR COLUNA DE UNIDADE PARA NOME ÚNICO DO HOSPITAL
df_producaoHRO2 = df_producaoHRO2.rename(columns={'Unidade': 'Hospital'})
df_producaoHRO2['Hospital'] = 'HOSPITAL 3'

In [ ]:
df_producaoHRO2.head(100)

# VISÃO -> ANO, MÊS, HOSP -> HOSPITAL 3

# % TOTAL DO HOSP NO TEMPO -> HOSPITAL 3

In [86]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_hro_AMH = df_producaoHRO2[
    (df_producaoHRO2['CID da Alta'].notna()) & 
    (df_producaoHRO2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_hro_AMH = df_producaoHRO2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_hro_AMH = df_cid_preenchido_hro_AMH.groupby(
    ['Ano', 'Mês', 'Hospital']
).agg(
    adm_com_cid_hro_AMH=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_hro_AMH = df_cid_total_hro_AMH.groupby(
    ['Ano', 'Mês', 'Hospital']
).agg(
    adm_total_hro_AMH=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHRO_AMH = pd.merge(
    agrupado_total_hro_AMH, 
    agrupado_cid_hro_AMH, 
    on=['Ano', 'Mês', 'Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHRO_AMH['% Admissões Com CID Preenchido'] = (
    (df_resultadoHRO_AMH['adm_com_cid_hro_AMH'] / df_resultadoHRO_AMH['adm_total_hro_AMH']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoHRO_AMH = df_resultadoHRO_AMH.rename(columns={'adm_total_hro_AMH': 'Total Admissões'})
df_resultadoHRO_AMH = df_resultadoHRO_AMH.rename(columns={'adm_com_cid_hro_AMH': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHRO_AMH['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHRO_AMH['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHRO_AMH.head(100)

# BAIXAR BASE / DF -> VISÃO ANO, MÊS, HOSP -> HOSPITAL 3

In [88]:
df_resultadoHRO_AMH.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> HOSP -> HOSPITAL 3

# % TOTAL DO HOSP -> HOSPITAL 3

In [89]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_hro_H = df_producaoHRO2[
    (df_producaoHRO2['CID da Alta'].notna()) & 
    (df_producaoHRO2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_hro_H = df_producaoHRO2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_hro_H = df_cid_preenchido_hro_H.groupby(
    ['Hospital']
).agg(
    adm_com_cid_hro_H=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_hro_H = df_cid_total_hro_H.groupby(
    ['Hospital']
).agg(
    adm_total_hro_H=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHRO_H = pd.merge(
    agrupado_total_hro_H, 
    agrupado_cid_hro_H, 
    on=['Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHRO_H['% Admissões Com CID Preenchido'] = (
    (df_resultadoHRO_H['adm_com_cid_hro_H'] / df_resultadoHRO_H['adm_total_hro_H']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoHRO_H = df_resultadoHRO_H.rename(columns={'adm_total_hro_H': 'Total Admissões'})
df_resultadoHRO_H = df_resultadoHRO_H.rename(columns={'adm_total_hro_H': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHRO_H['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHRO_H['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHRO_H.head(100)

# BAIXAR BASE / DF -> VISÃO HOSP -> HOSPITAL 3

In [91]:
df_resultadoHRO_H.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> ANO, MÊS, HOSP E MÉDICO -> HOSPITAL 3

# % TOTAL DO HOSP NO TEMPO POR MÉDICO -> HOSPITAL 3

In [92]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_hro_AMHM = df_producaoHRO2[
    (df_producaoHRO2['CID da Alta'].notna()) & 
    (df_producaoHRO2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_hro_AMHM = df_producaoHRO2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_hro_AMHM = df_cid_preenchido_hro_AMHM.groupby(
    ['Ano', 'Mês', 'Médico','Hospital']
).agg(
    adm_com_cid_hro_AMHM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_hro_AMHM = df_cid_total_hro_AMHM.groupby(
    ['Ano', 'Mês', 'Médico','Hospital']
).agg(
    adm_total_hro_AMHM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHRO_AMHM = pd.merge(
    agrupado_total_hro_AMHM, 
    agrupado_cid_hro_AMHM, 
    on=['Ano', 'Mês', 'Médico','Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHRO_AMHM['% Admissões Com CID Preenchido'] = (
    (df_resultadoHRO_AMHM['adm_com_cid_hro_AMHM'] / df_resultadoHRO_AMHM['adm_total_hro_AMHM']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoHRO_AMHM = df_resultadoHRO_AMHM.rename(columns={'adm_total_hro_AMHM': 'Total Admissões'})
df_resultadoHRO_AMHM = df_resultadoHRO_AMHM.rename(columns={'adm_com_cid_hro_AMHM': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHRO_AMHM['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHRO_AMHM['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHRO_AMHM.head(100)

# BAIXAR BASE / DF -> VISÃO ANO, MÊS, HOSP E MÉDICO -> HOSPITAL 3

In [94]:
df_resultadoHRO_AMHM.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> HOSP E MÉDICO -> HOSPITAL 3

# % TOTAL DO HOSP POR MÉDICO -> HOSPITAL 3

In [95]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_hro_HM = df_producaoHRO2[
    (df_producaoHRO2['CID da Alta'].notna()) & 
    (df_producaoHRO2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_hro_HM = df_producaoHRO2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_hro_HM = df_cid_preenchido_hro_HM.groupby(
    ['Médico','Hospital']
).agg(
    adm_com_cid_hro_HM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_hro_HM = df_cid_total_hro_HM.groupby(
    ['Médico','Hospital']
).agg(
    adm_total_hro_HM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHRO_HM = pd.merge(
    agrupado_total_hro_HM, 
    agrupado_cid_hro_HM, 
    on=['Médico','Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHRO_HM['% Admissões Com CID Preenchido'] = (
    (df_resultadoHRO_HM['adm_com_cid_hro_HM'] / df_resultadoHRO_HM['adm_total_hro_HM']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoHRO_HM = df_resultadoHRO_HM.rename(columns={'adm_total_hro_HM': 'Total Admissões'})
df_resultadoHRO_HM = df_resultadoHRO_HM.rename(columns={'adm_com_cid_hro_HM': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHRO_HM['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHRO_HM['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHRO_HM.head(100)

# BAIXAR BASE / DF -> VISÃO HOSP E MÉDICO -> HOSPITAL 3

In [97]:
df_resultadoHRO_HM.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# ///////////////////////////////////////////////////////////////////////////////////////////

# ///////////////////////////////////////////////////////////////////////////////////////////

# ///////////////////////////////////////////////////////////////////////////////////////////

# HOSPITAL 4

In [98]:
# PRODUÇÃO GERAL

# Especificar o caminho da pasta
folder_path = r" caminho da pasta para leitura de bases / arquivos excel e csv "

# Inicializar um DataFrame vazio
df_producaoOFTA = pd.DataFrame()

# Loop através de todos os arquivos na pasta
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    try:
        if file_name.endswith(('.xlsx', '.xls')):
            # Ler arquivo Excel
            df_temp = pd.read_excel(file_path)
        elif file_name.endswith('.csv'):
            # Tentar detectar automaticamente o delimitador
            df_temp = pd.read_csv(file_path, sep=None, engine='python')
        else:
            # Pular arquivos que não sejam Excel ou CSV
            continue
        
        # Concatenar os dados lidos com o DataFrame principal
        df_producaoOFTA = pd.concat([df_producaoOFTA, df_temp], ignore_index=True)
    except Exception as e:
        print(f"Erro ao processar o arquivo {file_name}: {e}")

df_producaoOFTA = df_producaoOFTA.drop_duplicates()
df_producaoOFTA['Cód. Paciente'] = df_producaoOFTA['Cód. Paciente'].astype(str)
df_producaoOFTA['Cód. Paciente'] = df_producaoOFTA['Cód. Paciente'].str.rstrip('.0')

df_producaoOFTA['Cód. Admissão'] = df_producaoOFTA['Cód. Admissão'].astype(str)
df_producaoOFTA['Cód. Admissão'] = df_producaoOFTA['Cód. Admissão'].str.rstrip('.0')

C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
df_producaoOFTA.head(100)

In [101]:
df_producaoOFTA['Cód. Admissão'].nunique()

84254

# TRATAR / FILTRAR DADOS -> HOSPITAL 4

In [102]:
df_producaoOFTA.columns

Index(['Ano', 'Mês', 'Cód. Admissão', 'Data Admissão', 'Hora Admissão',
       'Categoria', 'Subcategoria', 'Profissional Solicitante',
       'Profissional Admissão', 'Médico Externo', 'Classificação Produto',
       'CID da Alta', 'Tipo Recebimento', 'Convênio', 'Plano',
       'Perfil Particular', 'Procedimento Principal', 'Produto Lançamento',
       'Cód. Apresentação', 'Unidade', 'Especialidade', 'Caráter Admissão',
       'Tipo Paciente', 'Observação Admissão', 'Tipo Produto', 'Pacote',
       'Cód. Paciente', 'Paciente', 'Data Nascimento', 'Idade', 'Faixa Etária',
       'Destino', 'Qtd. Produzida', 'Valor Produzido'],
      dtype='object')

In [104]:
df_producaoOFTA2 = df_producaoOFTA.copy()

In [105]:
# TIRAR COLUNAS QUE NÃO VÃO SER UTILIZADAS
df_producaoOFTA2 = df_producaoOFTA2[['Cód. Paciente', 'Cód. Admissão', 'Ano', 'Mês', 'Profissional Admissão', 'Unidade', 'CID da Alta']]

# RENOMAR COLUNA DE UNIDADE PARA NOME ÚNICO DO HOSPITAL
df_producaoOFTA2 = df_producaoOFTA2.rename(columns={'Unidade': 'Hospital'})
df_producaoOFTA2['Hospital'] = 'HOSPITAL 4'

In [ ]:
df_producaoOFTA2.head(100)

# VISÃO -> ANO, MÊS, HOSP -> HOSPITAL 4

# % TOTAL DO HOSP NO TEMPO -> HOSPITAL 4

In [107]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_ofta_AMH = df_producaoOFTA2[
    (df_producaoOFTA2['CID da Alta'].notna()) & 
    (df_producaoOFTA2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_ofta_AMH = df_producaoOFTA2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_ofta_AMH = df_cid_preenchido_ofta_AMH.groupby(
    ['Ano', 'Mês', 'Hospital']
).agg(
    adm_com_cid_ofta_AMH=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_ofta_AMH = df_cid_total_ofta_AMH.groupby(
    ['Ano', 'Mês', 'Hospital']
).agg(
    adm_total_ofta_AMH=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoOFTA_AMH = pd.merge(
    agrupado_total_ofta_AMH, 
    agrupado_cid_ofta_AMH, 
    on=['Ano', 'Mês', 'Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoOFTA_AMH['% Admissões Com CID Preenchido'] = (
    (df_resultadoOFTA_AMH['adm_com_cid_ofta_AMH'] / df_resultadoOFTA_AMH['adm_total_ofta_AMH']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoOFTA_AMH = df_resultadoOFTA_AMH.rename(columns={'adm_total_ofta_AMH': 'Total Admissões'})
df_resultadoOFTA_AMH = df_resultadoOFTA_AMH.rename(columns={'adm_com_cid_ofta_AMH': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoOFTA_AMH['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoOFTA_AMH['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoOFTA_AMH.head(100)

# BAIXAR BASE / DF -> VISÃO ANO, MÊS, HOSP -> HOSPITAL 4

In [109]:
df_resultadoOFTA_AMH.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> HOSP -> HOSPITAL 4

# % TOTAL DO HOSP -> HOSPITAL 4

In [110]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_ofta_H = df_producaoOFTA2[
    (df_producaoOFTA2['CID da Alta'].notna()) & 
    (df_producaoOFTA2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_ofta_H = df_producaoOFTA2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_ofta_H = df_cid_preenchido_ofta_H.groupby(
    ['Hospital']
).agg(
    adm_com_cid_ofta_H=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_ofta_H = df_cid_total_ofta_H.groupby(
    ['Hospital']
).agg(
    adm_total_ofta_H=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoOFTA_H = pd.merge(
    agrupado_total_ofta_H, 
    agrupado_cid_ofta_H, 
    on=['Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoOFTA_H['% Admissões Com CID Preenchido'] = (
    (df_resultadoOFTA_H['adm_com_cid_ofta_H'] / df_resultadoOFTA_H['adm_total_ofta_H']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoOFTA_H = df_resultadoOFTA_H.rename(columns={'adm_total_ofta_H': 'Total Admissões'})
df_resultadoOFTA_H = df_resultadoOFTA_H.rename(columns={'adm_total_ofta_H': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoOFTA_H['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoOFTA_H['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoOFTA_H.head(100)

# BAIXAR BASE / DF -> VISÃO HOSP -> HOSPITAL 4

In [112]:
df_resultadoOFTA_H.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> ANO, MÊS, HOSP E MÉDICO -> HOSPITAL 4

# % TOTAL DO HOSP NO TEMPO POR MÉDICO -> HOSPITAL 4

In [113]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_ofta_AMHM = df_producaoOFTA2[
    (df_producaoOFTA2['CID da Alta'].notna()) & 
    (df_producaoOFTA2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_ofta_AMHM = df_producaoOFTA2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_ofta_AMHM = df_cid_preenchido_ofta_AMHM.groupby(
    ['Ano', 'Mês', 'Profissional Admissão','Hospital']
).agg(
    adm_com_cid_ofta_AMHM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_ofta_AMHM = df_cid_total_ofta_AMHM.groupby(
    ['Ano', 'Mês', 'Profissional Admissão','Hospital']
).agg(
    adm_total_ofta_AMHM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoOFTA_AMHM = pd.merge(
    agrupado_total_ofta_AMHM, 
    agrupado_cid_ofta_AMHM, 
    on=['Ano', 'Mês', 'Profissional Admissão','Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoOFTA_AMHM['% Admissões Com CID Preenchido'] = (
    (df_resultadoOFTA_AMHM['adm_com_cid_ofta_AMHM'] / df_resultadoOFTA_AMHM['adm_total_ofta_AMHM']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoOFTA_AMHM = df_resultadoOFTA_AMHM.rename(columns={'adm_total_ofta_AMHM': 'Total Admissões'})
df_resultadoOFTA_AMHM = df_resultadoOFTA_AMHM.rename(columns={'adm_com_cid_ofta_AMHM': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoOFTA_AMHM['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoOFTA_AMHM['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoOFTA_AMHM.head(100)

# BAIXAR BASE / DF -> VISÃO ANO, MÊS, HOSP E MÉDICO -> HOSPITAL 4

In [115]:
df_resultadoOFTA_AMHM.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> HOSP E MÉDICO -> HOSPITAL 4

# % TOTAL DO HOSP POR MÉDICO -> HOSPITAL 4

In [116]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_ofta_HM = df_producaoOFTA2[
    (df_producaoOFTA2['CID da Alta'].notna()) & 
    (df_producaoOFTA2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_ofta_HM = df_producaoOFTA2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_ofta_HM = df_cid_preenchido_ofta_HM.groupby(
    ['Profissional Admissão','Hospital']
).agg(
    adm_com_cid_ofta_HM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_ofta_HM = df_cid_total_ofta_HM.groupby(
    ['Profissional Admissão','Hospital']
).agg(
    adm_total_ofta_HM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoOFTA_HM = pd.merge(
    agrupado_total_ofta_HM, 
    agrupado_cid_ofta_HM, 
    on=['Profissional Admissão','Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoOFTA_HM['% Admissões Com CID Preenchido'] = (
    (df_resultadoOFTA_HM['adm_com_cid_ofta_HM'] / df_resultadoOFTA_HM['adm_total_ofta_HM']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoOFTA_HM = df_resultadoOFTA_HM.rename(columns={'adm_total_ofta_HM': 'Total Admissões'})
df_resultadoOFTA_HM = df_resultadoOFTA_HM.rename(columns={'adm_com_cid_ofta_HM': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoOFTA_HM['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoOFTA_HM['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoOFTA_HM.head(100)

# BAIXAR BASE / DF -> VISÃO HOSP E MÉDICO -> HOSPITAL 4

In [118]:
df_resultadoOFTA_HM.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# ///////////////////////////////////////////////////////////////////////////////////////////

# ///////////////////////////////////////////////////////////////////////////////////////////

# ///////////////////////////////////////////////////////////////////////////////////////////

# HOSPITAL 5

In [119]:
# PRODUÇÃO GERAL

# Especificar o caminho da pasta
folder_path = r" caminho da pasta para leitura de bases / arquivos excel e csv "

# Inicializar um DataFrame vazio
df_producaoHOSL = pd.DataFrame()

# Loop através de todos os arquivos na pasta
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    try:
        if file_name.endswith(('.xlsx', '.xls')):
            # Ler arquivo Excel
            df_temp = pd.read_excel(file_path)
        elif file_name.endswith('.csv'):
            # Tentar detectar automaticamente o delimitador
            df_temp = pd.read_csv(file_path, sep=None, engine='python')
        else:
            # Pular arquivos que não sejam Excel ou CSV
            continue
        
        # Concatenar os dados lidos com o DataFrame principal
        df_producaoHOSL = pd.concat([df_producaoHOSL, df_temp], ignore_index=True)
    except Exception as e:
        print(f"Erro ao processar o arquivo {file_name}: {e}")

df_producaoHOSL = df_producaoHOSL.drop_duplicates()
df_producaoHOSL['Cód. Paciente'] = df_producaoHOSL['Cód. Paciente'].astype(str)
df_producaoHOSL['Cód. Paciente'] = df_producaoHOSL['Cód. Paciente'].str.rstrip('.0')

df_producaoHOSL['Cód. Admissão'] = df_producaoHOSL['Cód. Admissão'].astype(str)
df_producaoHOSL['Cód. Admissão'] = df_producaoHOSL['Cód. Admissão'].str.rstrip('.0')

C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
df_producaoHOSL.head(100)

In [121]:
df_producaoHOSL['Cód. Admissão'].nunique()

353835

# TRATAR / FILTRAR DADOS -> HOSPITAL 5

In [122]:
df_producaoHOSL.columns

Index(['Ano', 'Mês', 'Cód. Admissão', 'Data Admissão', 'Hora Admissão',
       'Categoria', 'Subcategoria', 'Profissional Solicitante',
       'Profissional Admissão', 'Médico Externo', 'Classificação Produto',
       'CID da Alta', 'Tipo Recebimento', 'Convênio', 'Plano',
       'Perfil Particular', 'Procedimento Principal', 'Produto Lançamento',
       'Cód. Apresentação', 'Unidade', 'Especialidade', 'Caráter Admissão',
       'Tipo Paciente', 'Observação Admissão', 'Tipo Produto', 'Pacote',
       'Cód. Paciente', 'Paciente', 'Data Nascimento', 'Idade', 'Faixa Etária',
       'Destino', 'Qtd. Produzida', 'Valor Produzido', 'Solicitante', 'Médico',
       'Cirurgião', 'Origem Recebimento'],
      dtype='object')

In [123]:
df_producaoHOSL2 = df_producaoHOSL.copy()

In [124]:
# TIRAR COLUNAS QUE NÃO VÃO SER UTILIZADAS
df_producaoHOSL2 = df_producaoHOSL2[['Cód. Paciente', 'Cód. Admissão', 'Ano', 'Mês', 'Médico', 'Unidade', 'CID da Alta']]

# RENOMAR COLUNA DE UNIDADE PARA NOME ÚNICO DO HOSPITAL
df_producaoHOSL2 = df_producaoHOSL2.rename(columns={'Unidade': 'Hospital'})
df_producaoHOSL2['Hospital'] = 'HOSPITAL 5'

In [ ]:
df_producaoHOSL2.head(100)

# VISÃO -> ANO, MÊS, HOSP -> HOSPITAL 5

# % TOTAL DO HOSP NO TEMPO -> HOSPITAL 5

In [126]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_hosl_AMH = df_producaoHOSL2[
    (df_producaoHOSL2['CID da Alta'].notna()) & 
    (df_producaoHOSL2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_hosl_AMH = df_producaoHOSL2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_hosl_AMH = df_cid_preenchido_hosl_AMH.groupby(
    ['Ano', 'Mês', 'Hospital']
).agg(
    adm_com_cid_hosl_AMH=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_hosl_AMH = df_cid_total_hosl_AMH.groupby(
    ['Ano', 'Mês', 'Hospital']
).agg(
    adm_total_hosl_AMH=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHOSL_AMH = pd.merge(
    agrupado_total_hosl_AMH, 
    agrupado_cid_hosl_AMH, 
    on=['Ano', 'Mês', 'Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHOSL_AMH['% Admissões Com CID Preenchido'] = (
    (df_resultadoHOSL_AMH['adm_com_cid_hosl_AMH'] / df_resultadoHOSL_AMH['adm_total_hosl_AMH']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoHOSL_AMH = df_resultadoHOSL_AMH.rename(columns={'adm_total_hosl_AMH': 'Total Admissões'})
df_resultadoHOSL_AMH = df_resultadoHOSL_AMH.rename(columns={'adm_com_cid_hosl_AMH': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHOSL_AMH['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHOSL_AMH['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHOSL_AMH.head(100)

# BAIXAR BASE / DF -> VISÃO ANO, MÊS, HOSP -> HOSPITAL 5

In [128]:
df_resultadoHOSL_AMH.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> HOSP -> HOSPITAL 5

# % TOTAL DO HOSP -> HOSPITAL 5

In [129]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_hosl_H = df_producaoHOSL2[
    (df_producaoHOSL2['CID da Alta'].notna()) & 
    (df_producaoHOSL2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_hosl_H = df_producaoHOSL2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_hosl_H = df_cid_preenchido_hosl_H.groupby(
    ['Hospital']
).agg(
    adm_com_cid_hosl_H=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_hosl_H = df_cid_total_hosl_H.groupby(
    ['Hospital']
).agg(
    adm_total_hosl_H=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHOSL_H = pd.merge(
    agrupado_total_hosl_H, 
    agrupado_cid_hosl_H, 
    on=['Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHOSL_H['% Admissões Com CID Preenchido'] = (
    (df_resultadoHOSL_H['adm_com_cid_hosl_H'] / df_resultadoHOSL_H['adm_total_hosl_H']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoHOSL_H = df_resultadoHOSL_H.rename(columns={'adm_total_hosl_H': 'Total Admissões'})
df_resultadoHOSL_H = df_resultadoHOSL_H.rename(columns={'adm_total_hosl_H': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHOSL_H['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHOSL_H['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHOSL_H.head(100)

# BAIXAR BASE / DF -> VISÃO HOSP -> HOSPITAL 5

In [131]:
df_resultadoHOSL_H.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> ANO, MÊS, HOSP E MÉDICO -> HOSPITAL 5

# % TOTAL DO HOSP NO TEMPO POR MÉDICO -> HOSPITAL 5

In [132]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_hosl_AMHM = df_producaoHOSL2[
    (df_producaoHOSL2['CID da Alta'].notna()) & 
    (df_producaoHOSL2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_hosl_AMHM = df_producaoHOSL2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_hosl_AMHM = df_cid_preenchido_hosl_AMHM.groupby(
    ['Ano', 'Mês', 'Médico','Hospital']
).agg(
    adm_com_cid_hosl_AMHM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_hosl_AMHM = df_cid_total_hosl_AMHM.groupby(
    ['Ano', 'Mês', 'Médico','Hospital']
).agg(
    adm_total_hosl_AMHM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHOSL_AMHM = pd.merge(
    agrupado_total_hosl_AMHM, 
    agrupado_cid_hosl_AMHM, 
    on=['Ano', 'Mês', 'Médico','Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHOSL_AMHM['% Admissões Com CID Preenchido'] = (
    (df_resultadoHOSL_AMHM['adm_com_cid_hosl_AMHM'] / df_resultadoHOSL_AMHM['adm_total_hosl_AMHM']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoHOSL_AMHM = df_resultadoHOSL_AMHM.rename(columns={'adm_total_hosl_AMHM': 'Total Admissões'})
df_resultadoHOSL_AMHM = df_resultadoHOSL_AMHM.rename(columns={'adm_com_cid_hosl_AMHM': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHOSL_AMHM['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHOSL_AMHM['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHOSL_AMHM.head(100)

# BAIXAR BASE / DF -> VISÃO ANO, MÊS, HOSP E MÉDICO -> HOSPITAL 5

In [134]:
df_resultadoHOSL_AMHM.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> HOSP E MÉDICO -> HOSPITAL 5

# % TOTAL DO HOSP POR MÉDICO -> HOSPITAL 5

In [135]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_hosl_HM = df_producaoHOSL2[
    (df_producaoHOSL2['CID da Alta'].notna()) & 
    (df_producaoHOSL2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_hosl_HM = df_producaoHOSL2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_hosl_HM = df_cid_preenchido_hosl_HM.groupby(
    ['Médico','Hospital']
).agg(
    adm_com_cid_hosl_HM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_hosl_HM = df_cid_total_hosl_HM.groupby(
    ['Médico','Hospital']
).agg(
    adm_total_hosl_HM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHOSL_HM = pd.merge(
    agrupado_total_hosl_HM, 
    agrupado_cid_hosl_HM, 
    on=['Médico','Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHOSL_HM['% Admissões Com CID Preenchido'] = (
    (df_resultadoHOSL_HM['adm_com_cid_hosl_HM'] / df_resultadoHOSL_HM['adm_total_hosl_HM']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoHOSL_HM = df_resultadoHOSL_HM.rename(columns={'adm_total_hosl_HM': 'Total Admissões'})
df_resultadoHOSL_HM = df_resultadoHOSL_HM.rename(columns={'adm_com_cid_hosl_HM': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHOSL_HM['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHOSL_HM['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHOSL_HM.head(100)

# BAIXAR BASE / DF -> VISÃO HOSP E MÉDICO -> HOSPITAL 5

In [137]:
df_resultadoHOSL_HM.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# ///////////////////////////////////////////////////////////////////////////////////////////

# ///////////////////////////////////////////////////////////////////////////////////////////

# ///////////////////////////////////////////////////////////////////////////////////////////

# HOSPITAL 6

In [138]:
# PRODUÇÃO GERAL

# Especificar o caminho da pasta
folder_path = r" caminho da pasta para leitura de bases / arquivos excel e csv "

# Inicializar um DataFrame vazio
df_producaoHOLHOS = pd.DataFrame()

# Loop através de todos os arquivos na pasta
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    try:
        if file_name.endswith(('.xlsx', '.xls')):
            # Ler arquivo Excel
            df_temp = pd.read_excel(file_path)
        elif file_name.endswith('.csv'):
            # Tentar detectar automaticamente o delimitador
            df_temp = pd.read_csv(file_path, sep=None, engine='python')
        else:
            # Pular arquivos que não sejam Excel ou CSV
            continue
        
        # Concatenar os dados lidos com o DataFrame principal
        df_producaoHOLHOS = pd.concat([df_producaoHOLHOS, df_temp], ignore_index=True)
    except Exception as e:
        print(f"Erro ao processar o arquivo {file_name}: {e}")

df_producaoHOLHOS = df_producaoHOLHOS.drop_duplicates()
df_producaoHOLHOS['Cód. Paciente'] = df_producaoHOLHOS['Cód. Paciente'].astype(str)
df_producaoHOLHOS['Cód. Paciente'] = df_producaoHOLHOS['Cód. Paciente'].str.rstrip('.0')

df_producaoHOLHOS['Cód. Admissão'] = df_producaoHOLHOS['Cód. Admissão'].astype(str)
df_producaoHOLHOS['Cód. Admissão'] = df_producaoHOLHOS['Cód. Admissão'].str.rstrip('.0')


# CORRIGIR / UNIFICAR COLUNAS DE ANO e MÊS

# Preencher nulos de 'Ano' com valores de 'Data - Ano'
df_producaoHOLHOS['Ano'] = df_producaoHOLHOS['Ano'].fillna(df_producaoHOLHOS['Data - Ano'])

# Preencher nulos de 'Mês' com valores de 'Data - Mês'
df_producaoHOLHOS['Mês'] = df_producaoHOLHOS['Mês'].fillna(df_producaoHOLHOS['Data - Mês'])

# Remover linhas onde o valor de 'Ano' não é número
df_producaoHOLHOS = df_producaoHOLHOS[df_producaoHOLHOS['Ano'].astype(str).str.isnumeric()]

# Agora converter o tipo
df_producaoHOLHOS['Ano'] = df_producaoHOLHOS['Ano'].astype(int)

# Se quiser garantir que Mês seja texto
df_producaoHOLHOS['Mês'] = df_producaoHOLHOS['Mês'].astype(str)

# Remover colunas auxiliares
df_producaoHOLHOS.drop(columns=['Data - Ano', 'Data - Mês'], inplace=True)

C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site

In [ ]:
df_producaoHOLHOS.head(100)

In [148]:
df_producaoHOLHOS['Cód. Admissão'].nunique()

1441107

# TRATAR / FILTRAR DADOS -> HOSPITAL 6

In [149]:
df_producaoHOLHOS.columns

Index(['Cód. Admissão', 'Data Admissão', 'Hora Admissão', 'Categoria',
       'Subcategoria', 'Profissional Solicitante', 'Profissional Admissão',
       'Médico Externo', 'Classificação Produto', 'CID da Alta',
       'Tipo Recebimento', 'Origem Recebimento', 'Plano',
       'Procedimento Principal', 'Produto Lançamento', 'Cód. Apresentação',
       'Unidade', 'Especialidade', 'Caráter Admissão', 'Tipo Paciente',
       'Observação Admissão', 'Tipo Produto', 'Pacote', 'Cód. Paciente',
       'Paciente', 'Data Nascimento', 'Idade', 'Faixa Etária', 'Solicitante',
       'Médico', 'Cirurgião', 'Destino', 'Qtd. Produzida', 'Valor Produzido',
       'Ano', 'Mês', 'Convênio', 'Perfil Particular', 'Grupo Hospitalar'],
      dtype='object')

In [150]:
df_producaoHOLHOS2 = df_producaoHOLHOS.copy()

In [151]:
# TIRAR COLUNAS QUE NÃO VÃO SER UTILIZADAS
df_producaoHOLHOS2 = df_producaoHOLHOS2[['Cód. Paciente', 'Cód. Admissão', 'Ano', 'Mês', 'Médico', 'Unidade', 'CID da Alta']]

# RENOMAR COLUNA DE UNIDADE PARA NOME ÚNICO DO HOSPITAL
df_producaoHOLHOS2 = df_producaoHOLHOS2.rename(columns={'Unidade': 'Hospital'})
df_producaoHOLHOS2['Hospital'] = 'HOSPITAL 6'

In [ ]:
df_producaoHOLHOS2.head(100)

# VISÃO -> ANO, MÊS, HOSP -> HOSPITAL 6

# % TOTAL DO HOSP NO TEMPO -> HOSPITAL 6

In [153]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_holhos_AMH = df_producaoHOLHOS2[
    (df_producaoHOLHOS2['CID da Alta'].notna()) & 
    (df_producaoHOLHOS2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_holhos_AMH = df_producaoHOLHOS2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_holhos_AMH = df_cid_preenchido_holhos_AMH.groupby(
    ['Ano', 'Mês', 'Hospital']
).agg(
    adm_com_cid_holhos_AMH=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_holhos_AMH = df_cid_total_holhos_AMH.groupby(
    ['Ano', 'Mês', 'Hospital']
).agg(
    adm_total_holhos_AMH=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHOLHOS_AMH = pd.merge(
    agrupado_total_holhos_AMH, 
    agrupado_cid_holhos_AMH, 
    on=['Ano', 'Mês', 'Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHOLHOS_AMH['% Admissões Com CID Preenchido'] = (
    (df_resultadoHOLHOS_AMH['adm_com_cid_holhos_AMH'] / df_resultadoHOLHOS_AMH['adm_total_holhos_AMH']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoHOLHOS_AMH = df_resultadoHOLHOS_AMH.rename(columns={'adm_total_holhos_AMH': 'Total Admissões'})
df_resultadoHOLHOS_AMH = df_resultadoHOLHOS_AMH.rename(columns={'adm_com_cid_holhos_AMH': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHOLHOS_AMH['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHOLHOS_AMH['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHOLHOS_AMH.head(100)

# BAIXAR BASE / DF -> VISÃO ANO, MÊS, HOSP -> HOSPITAL 6

In [155]:
df_resultadoHOLHOS_AMH.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> HOSP -> HOSPITAL 6

# % TOTAL DO HOSP -> HOSPITAL 6

In [156]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_holhos_H = df_producaoHOLHOS2[
    (df_producaoHOLHOS2['CID da Alta'].notna()) & 
    (df_producaoHOLHOS2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_holhos_H = df_producaoHOLHOS2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_holhos_H = df_cid_preenchido_holhos_H.groupby(
    ['Hospital']
).agg(
    adm_com_cid_holhos_H=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_holhos_H = df_cid_total_holhos_H.groupby(
    ['Hospital']
).agg(
    adm_total_holhos_H=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHOLHOS_H = pd.merge(
    agrupado_total_holhos_H, 
    agrupado_cid_holhos_H, 
    on=['Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHOLHOS_H['% Admissões Com CID Preenchido'] = (
    (df_resultadoHOLHOS_H['adm_com_cid_holhos_H'] / df_resultadoHOLHOS_H['adm_total_holhos_H']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoHOLHOS_H = df_resultadoHOLHOS_H.rename(columns={'adm_total_holhos_H': 'Total Admissões'})
df_resultadoHOLHOS_H = df_resultadoHOLHOS_H.rename(columns={'adm_total_holhos_H': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHOLHOS_H['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHOLHOS_H['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHOLHOS_H.head(100)

# BAIXAR BASE / DF -> VISÃO HOSP -> HOSPITAL 6

In [158]:
df_resultadoHOLHOS_H.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> ANO, MÊS, HOSP E MÉDICO -> HOSPITAL 6

# % TOTAL DO HOSP NO TEMPO POR MÉDICO -> HOSPITAL 6

In [159]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_holhos_AMHM = df_producaoHOLHOS2[
    (df_producaoHOLHOS2['CID da Alta'].notna()) & 
    (df_producaoHOLHOS2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_holhos_AMHM = df_producaoHOLHOS2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_holhos_AMHM = df_cid_preenchido_holhos_AMHM.groupby(
    ['Ano', 'Mês', 'Médico','Hospital']
).agg(
    adm_com_cid_holhos_AMHM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_holhos_AMHM = df_cid_total_holhos_AMHM.groupby(
    ['Ano', 'Mês', 'Médico','Hospital']
).agg(
    adm_total_holhos_AMHM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHOLHOS_AMHM = pd.merge(
    agrupado_total_holhos_AMHM, 
    agrupado_cid_holhos_AMHM, 
    on=['Ano', 'Mês', 'Médico','Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHOLHOS_AMHM['% Admissões Com CID Preenchido'] = (
    (df_resultadoHOLHOS_AMHM['adm_com_cid_holhos_AMHM'] / df_resultadoHOLHOS_AMHM['adm_total_holhos_AMHM']) * 100
).round(2)

# Renomear colunas para melhor legibilidade
df_resultadoHOLHOS_AMHM = df_resultadoHOLHOS_AMHM.rename(columns={'adm_total_holhos_AMHM': 'Total Admissões'})
df_resultadoHOLHOS_AMHM = df_resultadoHOLHOS_AMHM.rename(columns={'adm_com_cid_holhos_AMHM': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHOLHOS_AMHM['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHOLHOS_AMHM['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHOLHOS_AMHM.head(100)

# BAIXAR BASE / DF -> VISÃO ANO, MÊS, HOSP E MÉDICO -> HOSPITAL 6

In [161]:
df_resultadoHOLHOS_AMHM.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> HOSP E MÉDICO -> HOSPITAL 6

# % TOTAL DO HOSP POR MÉDICO -> HOSPITAL 6

In [162]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_holhos_HM = df_producaoHOLHOS2[
    (df_producaoHOLHOS2['CID da Alta'].notna()) & 
    (df_producaoHOLHOS2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_holhos_HM = df_producaoHOLHOS2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_holhos_HM = df_cid_preenchido_holhos_HM.groupby(
    ['Médico','Hospital']
).agg(
    adm_com_cid_holhos_HM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_holhos_HM = df_cid_total_holhos_HM.groupby(
    ['Médico','Hospital']
).agg(
    adm_total_holhos_HM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHOLHOS_HM = pd.merge(
    agrupado_total_holhos_HM, 
    agrupado_cid_holhos_HM, 
    on=['Médico','Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHOLHOS_HM['% Admissões Com CID Preenchido'] = (
    (df_resultadoHOLHOS_HM['adm_com_cid_holhos_HM'] / df_resultadoHOLHOS_HM['adm_total_holhos_HM']) * 100
).round(2)

# Renomear colunas para melhor legibilidade
df_resultadoHOLHOS_HM = df_resultadoHOLHOS_HM.rename(columns={'adm_total_holhos_HM': 'Total Admissões'})
df_resultadoHOLHOS_HM = df_resultadoHOLHOS_HM.rename(columns={'adm_com_cid_holhos_HM': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHOLHOS_HM['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHOLHOS_HM['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHOLHOS_HM.head(100)

# BAIXAR BASE / DF -> VISÃO HOSP E MÉDICO -> HOSPITAL 6

In [164]:
df_resultadoHOLHOS_HM.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# ///////////////////////////////////////////////////////////////////////////////////////////

# ///////////////////////////////////////////////////////////////////////////////////////////

# ///////////////////////////////////////////////////////////////////////////////////////////

# HOSPITAL 7 

In [165]:
# PRODUÇÃO GERAL

# Especificar o caminho da pasta
folder_path = r" caminho da pasta para leitura de bases / arquivos excel e csv "

# Inicializar um DataFrame vazio
df_producaoHOPE = pd.DataFrame()

# Loop através de todos os arquivos na pasta
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    try:
        if file_name.endswith(('.xlsx', '.xls')):
            # Ler arquivo Excel
            df_temp = pd.read_excel(file_path)
        elif file_name.endswith('.csv'):
            # Tentar detectar automaticamente o delimitador
            df_temp = pd.read_csv(file_path, sep=None, engine='python')
        else:
            # Pular arquivos que não sejam Excel ou CSV
            continue
        
        # Concatenar os dados lidos com o DataFrame principal
        df_producaoHOPE = pd.concat([df_producaoHOPE, df_temp], ignore_index=True)
    except Exception as e:
        print(f"Erro ao processar o arquivo {file_name}: {e}")

df_producaoHOPE = df_producaoHOPE.drop_duplicates()
df_producaoHOPE['Cód. Paciente'] = df_producaoHOPE['Cód. Paciente'].astype(str)
df_producaoHOPE['Cód. Paciente'] = df_producaoHOPE['Cód. Paciente'].str.rstrip('.0')

df_producaoHOPE['Cód. Admissão'] = df_producaoHOPE['Cód. Admissão'].astype(str)
df_producaoHOPE['Cód. Admissão'] = df_producaoHOPE['Cód. Admissão'].str.rstrip('.0')

# CORRIGIR / UNIFICAR COLUNAS DE ANO e MÊS

# Preencher nulos de 'Ano' com valores de 'Data - Ano'
df_producaoHOPE['Ano'] = df_producaoHOPE['Ano'].fillna(df_producaoHOPE['Data - Ano'])

# Preencher nulos de 'Mês' com valores de 'Data - Mês'
df_producaoHOPE['Mês'] = df_producaoHOPE['Mês'].fillna(df_producaoHOPE['Data - Mês'])

# Remover linhas onde o valor de 'Ano' não é número
df_producaoHOPE = df_producaoHOPE[df_producaoHOPE['Ano'].astype(str).str.isnumeric()]

# Agora converter o tipo
df_producaoHOPE['Ano'] = df_producaoHOPE['Ano'].astype(int)

# Se quiser garantir que Mês seja texto
df_producaoHOPE['Mês'] = df_producaoHOPE['Mês'].astype(str)

# Remover colunas auxiliares
df_producaoHOPE.drop(columns=['Data - Ano', 'Data - Mês'], inplace=True)

C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\HenriquedaLuzPacheco\anaconda3\Lib\site

In [ ]:
df_producaoHOPE.head(100)

In [173]:
df_producaoHOPE['Cód. Admissão'].nunique()

1126624

# TRATAR / FILTRAR DADOS -> HOSPITAL 7

In [174]:
df_producaoHOPE.columns

Index(['Cód. Admissão', 'Data Admissão', 'Hora Admissão', 'Categoria',
       'Subcategoria', 'Profissional Solicitante', 'Profissional Admissão',
       'Médico Externo', 'Classificação Produto', 'CID da Alta',
       'Tipo Recebimento', 'Origem Recebimento', 'Plano',
       'Procedimento Principal', 'Produto Lançamento', 'Cód. Apresentação',
       'Unidade', 'Especialidade', 'Caráter Admissão', 'Tipo Paciente',
       'Observação Admissão', 'Tipo Produto', 'Pacote', 'Cód. Paciente',
       'Paciente', 'Data Nascimento', 'Idade', 'Faixa Etária', 'Solicitante',
       'Médico', 'Cirurgião', 'Destino', 'Qtd. Produzida', 'Valor Produzido',
       'Ano', 'Mês', 'Convênio', 'Perfil Particular', 'Grupo Hospitalar'],
      dtype='object')

In [176]:
df_producaoHOPE2 = df_producaoHOPE.copy()

In [177]:
# TIRAR COLUNAS QUE NÃO VÃO SER UTILIZADAS
df_producaoHOPE2 = df_producaoHOPE2[['Cód. Paciente', 'Cód. Admissão', 'Ano', 'Mês', 'Médico', 'Unidade', 'CID da Alta']]

# RENOMAR COLUNA DE UNIDADE PARA NOME ÚNICO DO HOSPITAL
df_producaoHOPE2 = df_producaoHOPE2.rename(columns={'Unidade': 'Hospital'})
df_producaoHOPE2['Hospital'] = 'HOSPITAL 7'

In [ ]:
df_producaoHOPE2.head(100)

# VISÃO -> ANO, MÊS, HOSP -> HOSPITAL 7

# % TOTAL DO HOSP NO TEMPO -> HOSPITAL 7

In [179]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_hope_AMH = df_producaoHOPE2[
    (df_producaoHOPE2['CID da Alta'].notna()) & 
    (df_producaoHOPE2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_hope_AMH = df_producaoHOPE2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_hope_AMH = df_cid_preenchido_hope_AMH.groupby(
    ['Ano', 'Mês', 'Hospital']
).agg(
    adm_com_cid_hope_AMH=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_hope_AMH = df_cid_total_hope_AMH.groupby(
    ['Ano', 'Mês', 'Hospital']
).agg(
    adm_total_hope_AMH=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHOPE_AMH = pd.merge(
    agrupado_total_hope_AMH, 
    agrupado_cid_hope_AMH, 
    on=['Ano', 'Mês', 'Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHOPE_AMH['% Admissões Com CID Preenchido'] = (
    (df_resultadoHOPE_AMH['adm_com_cid_hope_AMH'] / df_resultadoHOPE_AMH['adm_total_hope_AMH']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoHOPE_AMH = df_resultadoHOPE_AMH.rename(columns={'adm_total_hope_AMH': 'Total Admissões'})
df_resultadoHOPE_AMH = df_resultadoHOPE_AMH.rename(columns={'adm_com_cid_hope_AMH': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHOPE_AMH['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHOPE_AMH['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHOPE_AMH.head(100)

# BAIXAR BASE / DF -> VISÃO ANO, MÊS, HOSP -> HOSPITAL 7

In [181]:
df_resultadoHOPE_AMH.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> HOSP -> HOSPITAL 7

# % TOTAL DO HOSP -> HOSPITAL 7

In [182]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_hope_H = df_producaoHOPE2[
    (df_producaoHOPE2['CID da Alta'].notna()) & 
    (df_producaoHOPE2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_hope_H = df_producaoHOPE2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_hope_H = df_cid_preenchido_hope_H.groupby(
    ['Hospital']
).agg(
    adm_com_cid_hope_H=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_hope_H = df_cid_total_hope_H.groupby(
    ['Hospital']
).agg(
    adm_total_hope_H=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHOPE_H = pd.merge(
    agrupado_total_hope_H, 
    agrupado_cid_hope_H, 
    on=['Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHOPE_H['% Admissões Com CID Preenchido'] = (
    (df_resultadoHOPE_H['adm_com_cid_hope_H'] / df_resultadoHOPE_H['adm_total_hope_H']) * 100
).round(2)

# Renomar colunas para melhor legibilidade
df_resultadoHOPE_H = df_resultadoHOPE_H.rename(columns={'adm_total_hope_H': 'Total Admissões'})
df_resultadoHOPE_H = df_resultadoHOPE_H.rename(columns={'adm_com_cid_hope_H': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHOPE_H['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHOPE_H['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHOPE_H.head(100)

# BAIXAR BASE / DF -> VISÃO HOSP -> HOSPITAL 7

In [184]:
df_resultadoHOPE_H.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> ANO, MÊS, HOSP E MÉDICO -> HOSPITAL 7

# % TOTAL DO HOSP NO TEMPO POR MÉDICO -> HOSPITAL 7

In [185]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_hope_AMHM = df_producaoHOPE2[
    (df_producaoHOPE2['CID da Alta'].notna()) & 
    (df_producaoHOPE2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_hope_AMHM = df_producaoHOPE2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_hope_AMHM = df_cid_preenchido_hope_AMHM.groupby(
    ['Ano', 'Mês', 'Médico', 'Hospital']
).agg(
    adm_com_cid_hope_AMHM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_hope_AMHM = df_cid_total_hope_AMHM.groupby(
    ['Ano', 'Mês', 'Médico', 'Hospital']
).agg(
    adm_total_hope_AMHM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHOPE_AMHM = pd.merge(
    agrupado_total_hope_AMHM, 
    agrupado_cid_hope_AMHM, 
    on=['Ano', 'Mês', 'Médico', 'Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHOPE_AMHM['% Admissões Com CID Preenchido'] = (
    (df_resultadoHOPE_AMHM['adm_com_cid_hope_AMHM'] / df_resultadoHOPE_AMHM['adm_total_hope_AMHM']) * 100
).round(2)

# Renomear colunas para melhor legibilidade
df_resultadoHOPE_AMHM = df_resultadoHOPE_AMHM.rename(columns={'adm_total_hope_AMHM': 'Total Admissões'})
df_resultadoHOPE_AMHM = df_resultadoHOPE_AMHM.rename(columns={'adm_com_cid_hope_AMHM': 'Total Admissões Com CIDs Preenchidos'})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHOPE_AMHM['% Admissões Sem CID Preenchido'] = (
    (100 - df_resultadoHOPE_AMHM['% Admissões Com CID Preenchido'])).round(2)

In [ ]:
df_resultadoHOPE_AMHM.head(100)

# BAIXAR BASE / DF -> VISÃO ANO, MÊS, HOSP E MÉDICO -> HOSPITAL 7

In [187]:
df_resultadoHOPE_AMHM.to_excel(r" caminho para baixar a base resultado em excel ", index=False)

# VISÃO -> HOSP E MÉDICO -> HOSPITAL 7

# % TOTAL DO HOSP POR MÉDICO -> HOSPITAL 7

In [188]:
# ✅ DF 1: Admissões com CID preenchido (exclui 'SEM ALTA' e nulos)
df_cid_preenchido_hope_HM = df_producaoHOPE2[
    (df_producaoHOPE2['CID da Alta'].notna()) & 
    (df_producaoHOPE2['CID da Alta'].str.upper() != 'SEM ALTA')
]

# ✅ DF 2: Total de admissões (sem filtro)
df_cid_total_hope_HM = df_producaoHOPE2.copy()

# ✅ Agrupar DF 1: Total de admissões com CID preenchido
agrupado_cid_hope_HM = df_cid_preenchido_hope_HM.groupby(
    ['Médico', 'Hospital']
).agg(
    adm_com_cid_hope_HM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Agrupar DF 2: Total de admissões geral
agrupado_total_hope_HM = df_cid_total_hope_HM.groupby(
    ['Médico', 'Hospital']
).agg(
    adm_total_hope_HM=('Cód. Admissão', 'nunique')
).reset_index()

# ✅ Juntar os dois DataFrames
df_resultadoHOPE_HM = pd.merge(
    agrupado_total_hope_HM, 
    agrupado_cid_hope_HM, 
    on=['Médico', 'Hospital'], 
    how='left'
)

# ✅ Calcular percentual de admissões com CID preenchido
df_resultadoHOPE_HM['% Admissões Com CID Preenchido'] = (
    (df_resultadoHOPE_HM['adm_com_cid_hope_HM'] / df_resultadoHOPE_HM['adm_total_hope_HM']) * 100
).round(2)

# Renomear colunas para melhor legibilidade
df_resultadoHOPE_HM = df_resultadoHOPE_HM.rename(columns={
    'adm_total_hope_HM': 'Total Admissões',
    'adm_com_cid_hope_HM': 'Total Admissões Com CIDs Preenchidos'
})

# Criar coluna pegando a diferença de cids preenchidos com não preenchidos para deixar mais claro visivelmente
df_resultadoHOPE_HM['% Admissões Sem CID Preenchido'] = (
    100 - df_resultadoHOPE_HM['% Admissões Com CID Preenchido']
).round(2)

In [ ]:
df_resultadoHOPE_HM.head(100)

# BAIXAR BASE / DF -> VISÃO HOSP E MÉDICO -> HOSPITAL 7

In [190]:
df_resultadoHOPE_HM.to_excel(r" caminho para baixar a base resultado em excel ", index=False)